<a href="https://colab.research.google.com/github/AvichalTrivedi7/Generative-AI-Intel-Unnati/blob/main/hyper_parameter_tuning_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN)
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

---

##  Building and Tuning an ANN Model for Customer Churn Prediction

### **1️⃣ Data Preprocessing**

You started by preparing the dataset (`Churn_Modelling.csv`) for machine learning:

* **Dropped irrelevant columns:** `RowNumber`, `CustomerId`, `Surname`
* **Encoded categorical features:**

  * Used **LabelEncoder** for `Gender` → converted Male/Female to 0/1
  * Used **OneHotEncoder** for `Geography` → converted France/Germany/Spain into separate binary columns
* **Split data:** into features (`X`) and target (`y = Exited`)
* **Train-test split:** 80% for training, 20% for testing
* **Scaled features:** with `StandardScaler` for uniform numerical range
* **Saved preprocessing objects:** (`LabelEncoder`, `OneHotEncoder`, `Scaler`) using `pickle` for reuse during deployment

---

### **2️⃣ Creating a Model Builder Function**

You defined a flexible ANN builder function:

```python
def create_model(neurons=32, layers=1):
```

* Used **Sequential** model
* Added 1 or more **hidden layers** with ReLU activation
* Used a **sigmoid output layer** for binary classification
* Compiled the model with:

  * Optimizer: `adam`
  * Loss: `binary_crossentropy`
  * Metric: `accuracy`

Purpose: to dynamically create models with different architectures for hyperparameter tuning.

---

### **3️⃣ Wrapping the Model using `KerasClassifier`**

You created a scikit-learn-compatible ANN model:

```python
model = KerasClassifier(layers=1, neurons=32, build_fn=create_model, verbose=1)
```

* Enables usage of **GridSearchCV**
* Allows consistent `.fit()`, `.predict()`, and `.score()` behavior like any sklearn model

---

### **4️⃣ Defining the Hyperparameter Grid**

You specified combinations to explore:

```python
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}
```

This grid tells the search to test various neuron counts, network depths, and training durations.

---

### **5️⃣ Running Grid Search with Cross-Validation**

```python
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)
```

* **Performs exhaustive search** over all parameter combinations (16 total models × 3-fold CV)
* Uses parallel computation (`n_jobs=-1`)
* Finds the **best-performing architecture**
* Outputs best accuracy and parameters:

  ```python
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  ```

---

### ✅ **Overall Workflow Summary**

| Step                  | Purpose                                                        |
| --------------------- | -------------------------------------------------------------- |
| Data Cleaning         | Remove non-predictive identifiers                              |
| Encoding              | Convert categorical to numeric                                 |
| Scaling               | Standardize features for stable training                       |
| Model Definition      | Build flexible ANN via `create_model()`                        |
| Wrapping              | Use `KerasClassifier` to integrate with sklearn                |
| Hyperparameter Tuning | Use `GridSearchCV` to find optimal neurons, layers, and epochs |

---



In [ ]:
%pip install scikeras

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [ ]:
data=pd.read_csv('/content/Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [ ]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [ ]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [ ]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits



e:\UDemy Final\ANN Classification\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)



Epoch 1/100


250/250 [==============================] - 1s 1ms/step - loss: 0.6275 - accuracy: 0.6605
Epoch 2/100
250/250 [==============================] - 0s 958us/step - loss: 0.4631 - accuracy: 0.8009
Epoch 3/100
250/250 [==============================] - 0s 938us/step - loss: 0.4391 - accuracy: 0.8058
Epoch 4/100
250/250 [==============================] - 0s 972us/step - loss: 0.4250 - accuracy: 0.8140
Epoch 5/100
250/250 [==============================] - 0s 990us/step - loss: 0.4109 - accuracy: 0.8246
Epoch 6/100
250/250 [==============================] - 0s 961us/step - loss: 0.3973 - accuracy: 0.8310
Epoch 7/100
250/250 [==============================] - 0s 966us/step - loss: 0.3853 - accuracy: 0.8404
Epoch 8/100
250/250 [==============================] - 0s 978us/step - loss: 0.3755 - accuracy: 0.8457
Epoch 9/100
250/250 [==============================] - 0s 974us/step - loss: 0.3683 - accuracy: 0.8505
Epoch 10/100
250/250 [==============================] - 0s 967us/step - 